In [1]:
# @title Install InfluxDB Client
!pip install influxdb-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 745.8/745.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.8/217.8 kB 8.1 MB/s eta 0:00:00


In [24]:
# @title Import necessary modules
import influxdb_client
print("Influxdb_client Version : {}".format(influxdb_client.__version__))

from datetime import datetime
import numpy as np
%matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt

import time
from influxdb_client import InfluxDBClient
from IPython.display import display
import bokeh
print("Bokeh Version : {}".format(bokeh.__version__))
from bokeh.io import (output_notebook,
                      show,
                      push_notebook,
                      curdoc,
                      )
from bokeh.models import (ColumnDataSource,
                          DatetimeTickFormatter,
                          NumeralTickFormatter)
from bokeh.plotting import figure
import pandas as pd


#tables = query_api.query_data_frame(org='65c974ed52348dd6', query='from(bucket:"ardu-rasp") |> range(start: -1m) '
#'|> filter(fn: (r) => r._measurement == "ds18b20Sensors")')


Influxdb_client Version : 1.47.0
Bokeh Version : 3.6.1


In [6]:
# @title
client = InfluxDBClient(url='http://epics.ipfn.tecnico.ulisboa.pt:8086',
                        token='kJXa2Ok88yE8nSEyh2lStvs6v3vWYGhb90XL3M-gGhuysJWj36XaJkQaXSnGUa7h6FdkiIhZJ3JAYVCpc0390w==',
                        org='67c974ed52348dd6')
query_api = client.query_api()

In [34]:
# @title Live Temperature Data
output_notebook()
# Bokeh Time formatter
formatter = DatetimeTickFormatter(
    microseconds = "%H:%M:%S",
    milliseconds = "%H:%M:%S",
    seconds = "%H:%M:%S",
    minsec = "%H:%M:%S",
    minutes = "%H:%M:%S",
    hourmin = "%H:%M:%S",
    hours= "%H:%M:%S",
    days= "%H:%M:%S",
    months="%H:%M:%S",
    years="%H:%M:%S",
)
ACQ_TIME = 240 # in sec
LIVE_INTERVAL = 10 # in min
query_temp = (f'from(bucket:"ardu-rasp") |> range(start: -{LIVE_INTERVAL}m) '
    '|> filter(fn: (r) => r._measurement == "ds18b20Sensors") '
    '|> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")')
temperature_table_df = query_api.query_data_frame(org='67c974ed52348dd6', query=query_temp)
source = ColumnDataSource(temperature_table_df)
curdoc().theme = "dark_minimal"
fig = figure(title="ARDUINO-RASP Live Temperature Plot",
            x_axis_type="datetime",
            sizing_mode="stretch_width",
            x_axis_label='Time',
            y_axis_label='ºC',
            y_range=(10, 50),)

fig.line(x='_time', y='tempBlue', source=source, color="blue", line_width=2)
fig.line(x='_time', y='tempRed', source=source, color="red", line_width=2)
fig.xaxis[0].formatter = formatter  # DatetimeTickFormatter() # months="%b %Y")
t=show(fig, notebook_handle=True)

def update_plot():
    temperature_table_df = query_api.query_data_frame(org='67c974ed52348dd6', query=query_temp)
    source.data = temperature_table_df
    # source.stream(new_tables_df)
    push_notebook(handle=t)

for i in range(ACQ_TIME):
    update_plot()
    time.sleep(1)

KeyboardInterrupt: 

# Calculations

In [35]:
# @title Get saved Data
RANGE = 'start:2024-11-15T14:28:00Z, stop:2024-11-15T14:40:00Z'
output_notebook()
query_saved_temp = (f'from(bucket:"ardu-rasp") |> range({RANGE}) '
    '|> filter(fn: (r) => r._measurement == "ds18b20Sensors") '
    '|> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")')
query_api = client.query_api()
ft2 = DatetimeTickFormatter(
    milliseconds = "%H:%M:%S",
    seconds = "%H:%M:%S",
    minutes = "%H:%M:%S"
)
temperature_table_df = query_api.query_data_frame(org='67c974ed52348dd6', query=query_saved_temp)
source = ColumnDataSource(temperature_table_df)
fig = figure(title="ARDUINO-RASP Temperature Plot",
            x_axis_type="datetime",
            sizing_mode="stretch_width",
            x_axis_label='Time',
            y_axis_label='ºC',
            y_range=(10, 50),)

fig.line(x='_time', y='tempBlue', source=source, color="blue", line_width=2)
fig.line(x='_time', y='tempRed', source=source, color="red", line_width=2)
fig.xaxis[0].formatter = ft2

t=show(fig, notebook_handle=True)


In [30]:
# @title Interactive Plot
tRed = np.array(temperature_table_df['tempRed'])
tBlue = np.array(temperature_table_df['tempBlue'])
npoints = tRed.shape[0]

def f(x0):
    plt.figure(2)
    plt.plot(tRed, label='Red')
    plt.plot(tBlue, label='Blue')
    plt.legend()
    x = np.array([x0, x0]) # np.linspace(-10, 10, num=1000)
    y = np.array([0, 100]) # np.linspace(-10, 10, num=1000)
    plt.plot(x, y)
    # plt.plot(x, m * x + b)
    plt.xlim(0, npoints)
    plt.ylim(0, 100)
    plt.show()
    tRedx0 = tRed[x0]
    tBluex0 = tBlue[x0]
    print(f'Temp Red: {tRedx0}, Temp Blue: {tBluex0}')

interactive_plot = interactive(f, x0=(0, npoints, 1)) #  , b=(-3, 3, 0.5))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot


interactive(children=(IntSlider(value=240, description='x0', max=481), Output(layout=Layout(height='350px'))),…

In [33]:
# @title Save CSV File (You may need to download file from 'Files' section)
T_mat = np.array([tRed, tBlue])
filename = f"Temperature_log_{datetime.now():%Y_%m_%d_%H-%M-%S}.csv"
np.savetxt(filename, T_mat.T, delimiter=",")